# Golden test: comparación de cargas y cálculos
Este notebook carga los JSON exportados desde Laravel (`equipments.json`, `invoices.json`), reconstruye los cálculos según la lógica Python (factores de carga y eficiencia) y escribe un reporte de discrepancias en `errores_golden_test.txt`.

Rutas esperadas (relativas a este archivo): `../storage/app/private/exports/equipments.json` y `../storage/app/private/exports/invoices.json`.

In [ ]:
# 1) Importar librerías
import pandas as pd
import numpy as np
import json
import os
from datetime import datetime

pd.set_option('display.max_columns', None)
print('Librerías cargadas')

In [ ]:
# 2) Cargar los JSON exportados desde Laravel
base_exports = os.path.join('..', 'storage', 'app', 'private', 'exports')
equipments_path = os.path.join(base_exports, 'equipments.json')
invoices_path = os.path.join(base_exports, 'invoices.json')

print('Ruta equipments:', equipments_path)
print('Ruta invoices:', invoices_path)

# Cargar con tolerancia a errores
if not os.path.exists(equipments_path):
    raise FileNotFoundError(f'No se encuentra {equipments_path} — exportá los JSON desde Laravel en storage/app/private/exports')

with open(equipments_path, 'r', encoding='utf-8') as f:
    equipments_json = json.load(f)

equipments_df = pd.DataFrame(equipments_json)
print(f'Equipos cargados: {len(equipments_df)}')

# Cargar facturas si existen
invoices_df = pd.DataFrame()
if os.path.exists(invoices_path):
    with open(invoices_path, 'r', encoding='utf-8') as f:
        invoices_json = json.load(f)
    invoices_df = pd.DataFrame(invoices_json)
    print(f'Facturas cargadas: {len(invoices_df)}')
else:
    print('No se encontró invoices.json — algunas comparaciones monetarias no serán posibles')

In [ ]:
# 3) Definir la tabla de factores (igual que en el script Python)
data = {
    'tipo de proceso': ['Motor', 'Resistencia', 'Electrónico', 'Motor & Resistencia', 'Magnetrón', 'Electroluminiscencia'],
    'factor_carga': [0.7, 1, 0.7, 0.8, 0.7, 1],
    'eficiencia': [0.9, 0.6, 0.8, 0.82, 0.6, 0.9]
}
fqe_df = pd.DataFrame(data)
fqe_df

In [ ]:
# 4) Combinar factores con los equipos (si existe 'tipo de proceso')
df = equipments_df.copy()
if 'tipo de proceso' in df.columns:
    df = pd.merge(df, fqe_df, on='tipo de proceso', how='left')
    missing = df[df['factor_carga'].isna()]['tipo de proceso'].unique()
    if len(missing):
        print('Faltan factores para tipos de proceso:', missing)
        # Rellenar con defaults razonables
        df['factor_carga'] = df['factor_carga'].fillna(1)
        df['eficiencia'] = df['eficiencia'].fillna(1)
else:
    print(
)
    df['factor_carga'] = 1
    df['eficiencia'] = 1

# Normalizar columnas esperadas
for col in ['potencia_watts', 'cantidad', 'horas_por_dia', 'standby_watts']:
    if col not in df.columns:
        df[col] = 0

df[['nombre','categoria','ubicacion','cantidad','potencia_watts','horas_por_dia','factor_carga','eficiencia']].head()

In [ ]:
# 5) Calcular energías (Wh) según la lógica del script Python
# EnergiaConsumida_Wh = horas_por_dia * factor_carga * cantidad * potencia_watts / eficiencia
# EnergiaUtilConsumida_Wh = horas_por_dia * factor_carga * cantidad * potencia_watts * eficiencia
df['energia_consumida_wh'] = df['horas_por_dia'] * df['factor_carga'] * df['cantidad'] * df['potencia_watts'] / df['eficiencia'].replace(0, np.nan)
df['energia_util_consumida_wh'] = df['horas_por_dia'] * df['factor_carga'] * df['cantidad'] * df['potencia_watts'] * df['eficiencia']

# Llenar NaNs si aparecieron (por eficiencia 0)
df['energia_consumida_wh'] = df['energia_consumida_wh'].fillna(0)
df['energia_util_consumida_wh'] = df['energia_util_consumida_wh'].fillna(0)

df[['nombre','energia_consumida_wh','energia_util_consumida_wh']].head()

In [ ]:
# 6) Calcular por período usando la primera factura (si está disponible)
if not invoices_df.empty:
    f0 = invoices_df.iloc[0].to_dict()
    fecha_inicio = datetime.strptime(f0['fecha_inicio'], '%Y-%m-%d')
    fecha_fin = datetime.strptime(f0['fecha_fin'], '%Y-%m-%d')
    dias_en_periodo = (fecha_fin - fecha_inicio).days
    tarifa_promedio = f0.get('tarifa_promedio', None)
    print(f'Días en periodo: {dias_en_periodo}, tarifa_promedio: {tarifa_promedio}')
else:
    dias_en_periodo = None
    tarifa_promedio = None
    print('No hay facturas para calcular periodo')

if dias_en_periodo is not None:
    df['energia_consumida_wh_periodo'] = df['energia_consumida_wh'] * dias_en_periodo
    # costo requiere kWh -> convertir Wh a kWh para el periodo
    if tarifa_promedio is not None:
        df['costo_monetario_periodo'] = (df['energia_consumida_wh_periodo'] / 1000.0) * tarifa_promedio
    else:
        df['costo_monetario_periodo'] = np.nan
else:
    df['energia_consumida_wh_periodo'] = np.nan
    df['costo_monetario_periodo'] = np.nan

df[['nombre','energia_consumida_wh_periodo','costo_monetario_periodo']].head()

In [ ]:
# 7) Comparar con campos exportados y registrar discrepancias
errores = []
tolerancia_wh = 1e-6  # tolerancia absoluta en Wh (ajustable)

# Normalizar nombres de columnas exportadas esperadas
# Posibles campos en el JSON exportado: kwh_activo_año, kwh_standby_año, kwh_total_año (en kWh por año)
if 'kwh_total_año' in df.columns:
    # comparar valores anuales (convertir nuestro cálculo a anual en kWh)
    df['computed_kwh_anual'] = (df['energia_consumida_wh'] * 365.0) / 1000.0
    # si el JSON tiene kwh_total_año ya en kWh, comparo directamente
    for idx, row in df.iterrows():
        export_val = row.get('kwh_total_año', None)
        comp_val = row.get('computed_kwh_anual', None)
        if pd.notna(export_val) and pd.notna(comp_val):
            diff = float(export_val) - float(comp_val)
            if abs(diff) > (0.01):  # tolerancia en kWh anual (ajustable)
                errores.append(f'Equipo: {row.get("nombre")} | kwh_total_año | exportado: {export_val} | calculado: {comp_val:.3f} | diff: {diff:.3f}')

# Comparación por período si fue calculado
if 'energia_consumida_wh_periodo' in df.columns and dias_en_periodo is not None:
    df['computed_kwh_periodo'] = df['energia_consumida_wh_periodo'] / 1000.0
    # buscar campo de export si existe (por ejemplo, si el export incluye un campo de periodo)
    # aquí intentamos comparar con 'kwh_activo_año' / 365*dias etc.
    for idx, row in df.iterrows():
        comp_kwh_periodo = row.get('computed_kwh_periodo', None)
        # si el export tiene horas por año podemos construir un periodo comparable; si no, lo registramos
        # Buscar 'kwh_activo_año' para comparar escala anual
        export_kwh_activo_anual = row.get('kwh_activo_año', None)
        if pd.notna(export_kwh_activo_anual) and pd.notna(comp_kwh_periodo):
            # convertir export a kWh del mismo periodo: export_kwh_activo_anual * (dias_en_periodo / 365)
            export_kwh_periodo = float(export_kwh_activo_anual) * (dias_en_periodo / 365.0)
            diff = export_kwh_periodo - float(comp_kwh_periodo)
            if abs(diff) > 0.01:
                errores.append(f'Equipo: {row.get("nombre")} | periodo kWh | exportado(periodo): {export_kwh_periodo:.3f} | calculado(periodo): {comp_kwh_periodo:.3f} | diff: {diff:.3f}')

# Escribir errores en archivo si los hubo
errores_path = os.path.join(base_exports, 'errores_golden_test.txt')
if errores:
    with open(errores_path, 'w', encoding='utf-8') as f:
        f.write('# Errores de comparación golden test

')
        for e in errores:
            f.write(e + '
')
    print(f'Se registraron {len(errores)} discrepancias en: {errores_path}')
else:
    print('No se encontraron discrepancias relevantes')

## Instrucciones rápidas para usar este notebook
1. Asegurate de ejecutar las exportaciones de Laravel: `php artisan inventory:export-usage` y `php artisan invoices:export`.
2. Colocar los archivos `equipments.json` e `invoices.json` en `storage/app/private/exports/` del repositorio.
3. Ejecutar las celdas de arriba en orden.
4. Revisar `storage/app/private/exports/errores_golden_test.txt` para ver discrepancias detectadas.

Si querés que el notebook compare más campos (standby, carga por ubicación, etc.) lo podemos ampliar.